In [78]:
import pandas as pd
import numpy as np
from random import shuffle
from osgeo import ogr, osr
from sentinelhub import WmsRequest, WcsRequest, MimeType, CRS, BBox, constants, DataSource, CustomUrlParam
from s2cloudless import S2PixelCloudDetector, CloudMaskRequest
import logging
from collections import Counter
import datetime
import os
import yaml

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from skimage.transform import resize

import scipy.sparse as sparse
from scipy.sparse.linalg import splu


with open("../../config.yaml", 'r') as stream:
        key = (yaml.safe_load(stream))
        API_KEY = key['key'] 
        
%run ../../src/slope.py
%run ../../src/utils-bilinear.py
%run ../../src/dsen2/utils/DSen2Net.py

In [88]:
df = pd.read_csv("../../data/test-csv/fao-test-shifted.csv")
df['PLOT_ID'] = range(0, len(df))

In [80]:
time = ('2018-12-15', '2020-01-15')
EPSG = CRS.WGS84
IMSIZE = 48
cloud_detector = S2PixelCloudDetector(threshold=0.4, average_over=4, dilation_size=2)

# Helper functions (to be moved to a utils file)

In [81]:
def calculate_proximal_steps_index(date, satisfactory):
    arg_before = None
    arg_after = None
    if date > 0:
        idx_before = satisfactory - date
        arg_before = idx_before[np.where(idx_before < 0, idx_before, -np.inf).argmax()]
    if date < np.max(satisfactory):
        idx_after = satisfactory - date
        arg_after = idx_after[np.where(idx_after > 0, idx_after, np.inf).argmin()]
    if not arg_after and not arg_before:
        arg_after = date
        arg_before = date
    if not arg_after:
        arg_after = arg_before
    if not arg_before:
        arg_before = arg_after
    return arg_before, arg_after

def speyediff(N, d, format = 'csc'):
    shape = (N-d, N)
    diagonals = np.zeros(2*d + 1)
    diagonals[d] = 1.
    for i in range(d):
        diff = diagonals[:-1] - diagonals[1:]
        diagonals = diff
    offsets = np.arange(d+1)
    spmat = sparse.diags(diagonals, offsets, shape, format = format)
    return spmat

def smooth(y, lmbd, d = 2):
    m = len(y)
    E = sparse.eye(m, format = 'csc')
    D = speyediff(m, d, format = 'csc')
    coefmat = E + lmbd * D.conj().T.dot(D)
    z = splu(coefmat).solve(y)
    return z

MDL_PATH = "../../src/dsen2/models/"

input_shape = ((4, None, None), (6, None, None))
model = s2model(input_shape, num_layers=6, feature_size=128)
predict_file = MDL_PATH+'s2_032_lr_1e-04.hdf5'
print('Symbolic Model Created.')

model.load_weights(predict_file)

def DSen2(d10, d20):
    test = [d10, d20]
    input_shape = ((4, None, None), (6, None, None))
    prediction = _predict(test, input_shape, deep=False)
    #prediction *= 5
    return prediction

def _predict(test, input_shape, model = model, deep=False, run_60=False):
    
    print("Predicting using file: {}".format(predict_file))
    prediction = model.predict(test, verbose=1)
    return prediction

c_arr = np.array([[1, 1, 1, 1, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 2, 3, 2, 1,],
                  [1, 2, 2, 2, 1,],
                  [1, 1, 1, 1, 1,],])
                  
c_arr = c_arr / 3
o_arr = 1 - c_arr
c_arr = np.tile(c_arr[:, :, np.newaxis], (1, 1, 11))
o_arr = np.tile(o_arr[:, :, np.newaxis], (1, 1, 11))

Symbolic Model Created.


# Bounding boxes

In [90]:
def convertCoords(xy, src='', targ=''):

    srcproj = osr.SpatialReference()
    srcproj.ImportFromEPSG(src)
    targproj = osr.SpatialReference()
    if isinstance(targ, str):
        targproj.ImportFromProj4(targ)
    else:
        targproj.ImportFromEPSG(targ)
    transform = osr.CoordinateTransformation(srcproj, targproj)

    pt = ogr.Geometry(ogr.wkbPoint)
    pt.AddPoint(xy[0], xy[1])
    pt.Transform(transform)
    return([pt.GetX(), pt.GetY()])


def calculate_epsg(points):
    lon, lat = points[0], points[1]
    print(lon, lat)
    utm_band = str((math.floor((lon + 180) / 6 ) % 60) + 1)
    if len(utm_band) == 1:
        utm_band = '0'+utm_band
    if lat >= 0:
        epsg_code = '326' + utm_band
    else:
        epsg_code = '327' + utm_band
    return int(epsg_code)

def calc_bbox(plot_id, df):
    print(plot_id)
    subs = df[df['PLOTID'] == plot_id]
    # TOP, LEFT, BOTTOM, RIGHT
    # (min x, min y), (max x, max y)
    return [(min(subs['LON']), min(subs['LAT'])),
            (max(subs['LON']), max(subs['LAT']))]

def bounding_box(points, expansion = 160):
    # LONG, LAT FOR SOME REASON
    middle = points  
    epsg = calculate_epsg(middle)

    middle = convertCoords(middle, 4326, epsg)
    expansion1 = 75
    expansion2 = 85
    bl = [middle[0] - 235, middle[1] - 235]
    tr = [middle[0] + 245, middle[1] + 245]

    after = [b - a for a,b in zip(bl, tr)]   
    print(np.array(bl) - np.array(middle))
    print(np.array(tr) - np.array(middle))
    
    bl = convertCoords(bl, epsg, 4326)
    tr = convertCoords(tr, epsg, 4326)
    return bl, tr

# Data download

In [83]:
def rank_array(arr):
    order = arr.argsort()
    ranks = order.argsort()
    return ranks

def mcm_shadow_mask(arr, c_probs):
    #' From "Cloud and cloud shadow masking for Sentinel-2 using multi-
    #         temporal images in global area. Candra et al. 2020."
    #'[B02,B03,B04, B08, B05,B06,B07, B8A,B11,B12]
    #'[B01,B02,B04,B05,B08,B8A,B09,B10,B11,B12]
    #'[ 0 , 1 , 2 , 3  , 4  , 5,  6   7  , 8  , 9]'
    mean_c_probs = np.mean(c_probs, axis = (1, 2))
    cloudy_steps = np.argwhere(mean_c_probs > 0.25)
    images_clean = np.delete(arr, cloudy_steps, 0)
    cloud_ranks = rank_array(mean_c_probs)
    diffs = abs(np.sum(arr - np.mean(images_clean, axis = 0), axis = (1, 2, 3)))
    diff_ranks = rank_array(diffs)
    overall_rank = diff_ranks + cloud_ranks
    reference_idx = np.argmin(overall_rank)
    ri = arr[reference_idx]
    print(reference_idx)
    
    nir_means = np.mean(arr[:, :, :, 4], axis = (0))
    
    shadows = np.zeros((arr.shape[0], 96, 96))    
    # Candra et al. 2020
    
    for time in range(arr.shape[0]):
        for x in range(arr.shape[1]):
            for y in range(arr.shape[2]):
                ti_slice = arr[time, x, y]
                ri_slice = ri[x, y]
                deltab2 = ti_slice[1] - ri_slice[1]
                deltab3 = ti_slice[2] - ri_slice[2]
                deltab4 = ti_slice[3] - ri_slice[3]
                deltab8a = ti_slice[6] - ri_slice[6]
                deltab11 = ti_slice[8] - ri_slice[8]

                if deltab2 < 0.1:
                    if deltab3 < 0.08:
                        if deltab4 < 0.08:
                            if deltab8a < -0.04:
                                if deltab11 < -0.04:
                                    if ti_slice[1] < 0.0950:
                                        shadows[time, x, y] = 1.
    print(np.sum(shadows, axis = (1, 2)))
                                        
    # Additional 3 time-window NIR thresholding
    
    for time in range(shadows.shape[0]):
        for x in range(shadows.shape[1]):
            for y in range(shadows.shape[2]):
                if shadows[time, x, y] == 1:
                    time_adjusted = np.max([time, 1])
                    time_adjusted = np.min([time_adjusted, shadows.shape[0] - 2])
                    # check before and after
                    if shadows[time_adjusted + 1, x, y] == 1 or shadows[time_adjusted-1, x, y] == 1:
                        nir_mean = nir_means[x, y]
                        nir_before = arr[time_adjusted - 1, x, y, 4] - nir_mean
                        nir_now = arr[time, x, y, 4] - nir_mean
                        nir_next = arr[time_adjusted + 1, x, y, 4] - nir_mean
                        if nir_now > nir_mean * 0.75:
                            shadows[time, x, y] = 0
                        if nir_next or nir_before <= nir_now:
                            shadows[time, x, y] = 0
                         
    print(np.sum(shadows, axis = (1, 2)))
    
    # Remove shadows if cannot coreference a cloud
    shadow_large = np.reshape(shadows, (shadows.shape[0], 96//8, 8, 96//8, 8))
    shadow_large = np.sum(shadow_large, axis = (2, 4))
    
    cloud_large = np.copy(c_probs)
    cloud_large[np.where(c_probs > 0.33)] = 1.
    cloud_large[np.where(c_probs < 0.33)] = 0.
    cloud_large = np.reshape(cloud_large, (shadows.shape[0], 96//8, 8, 96//8, 8))
    cloud_large = np.sum(cloud_large, axis = (2, 4))
    for time in range(shadow_large.shape[0]):
        for x in range(shadow_large.shape[1]):
            x_low = np.max([x - 8, 0])
            x_high = np.min([x + 8, shadow_large.shape[1] - 1])
            for y in range(shadow_large.shape[2]):
                y_low = np.max([y - 8, 0])
                y_high = np.min([y + 8, shadow_large.shape[1] - 1])
                if shadow_large[time, x, y] < 8:
                    shadow_large[time, x, y] = 0.
                if shadow_large[time, x, y] >= 8:
                    shadow_large[time, x, y] = 1.
                c_prob_window = cloud_large[time, x_low:x_high, y_low:y_high]
                if np.max(c_prob_window) < 24:
                    shadow_large[time, x, y] = 0.
                    
    
    shadow_large = resize(shadow_large, (shadow_large.shape[0], 96, 96), order = 0)
    shadows *= shadow_large
    print(np.sum(shadows, axis = (1, 2)))
    
    # Go through and aggregate the shadow map to an 80m grid, and extend it one grid size around
    # any positive ID
    
    shadows_original = np.copy(shadows)
    shadows = np.reshape(shadows, (shadows.shape[0], 96//8, 8, 96//8, 8))
    shadows = np.sum(shadows, axis = (2, 4))
    shadows[np.where(shadows < 16)] = 0.
    shadows[np.where(shadows >= 16)] = 1.
    shadows = resize(shadows, (shadows.shape[0], 96, 96), order = 0)
    shadows = np.reshape(shadows, (shadows.shape[0], 96//4, 4, 96//4, 4))
    shadows = np.max(shadows, (2, 4))
    
    shadows_new = np.zeros_like(shadows)
    for time in range(shadows.shape[0]):
        for x in range(shadows.shape[1]):
            for y in range(shadows.shape[2]):
                if shadows[time, x, y] == 1:
                    min_x = np.max([x - 1, 0])
                    max_x = np.min([x + 2, shadows.shape[1] - 1])
                    min_y = np.max([y - 1, 0])
                    max_y = np.min([y + 2, shadows.shape[1] - 1])
                    for x_idx in range(min_x, max_x):
                        for y_idx in range(min_y, max_y):
                            shadows_new[time, x_idx, y_idx] = 1.
    shadows_new = resize(shadows_new, (shadows.shape[0], 96, 96), order = 0)
    print("The shadow probability is: {}".format(100*np.sum(shadows_new)/(96*96*shadows_new.shape[0])))
    print(np.sum(shadows_new, axis = (1, 2)))
    return shadows_new, shadows_original

def identify_clouds(bbox, epsg = EPSG, time = time):
    #try:
    box = BBox(bbox, crs = epsg)
    cloud_request = WmsRequest(
        layer='CLOUD_DETECTION',
        bbox=box,
        time=time,
        width=96,
        height=96,
        image_format = MimeType.TIFF_d32f,
        maxcc=0.75,
        instance_id=API_KEY,
        custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
        time_difference=datetime.timedelta(hours=48),
    )

    cloud_img = cloud_request.get_data()
    print(np.array(cloud_img).shape)
    cloud_probs = cloud_detector.get_cloud_probability_maps(np.array(cloud_img))
    shadows, shadow_original = mcm_shadow_mask(np.array(cloud_img), cloud_probs)
    shadows = shadows[:, 24:-24, 24:-24]
    return cloud_probs[:, 24:-24, 24:-24], shadows, shadow_original, cloud_probs
    #except Exception as e:
    #    logging.fatal(e, exc_info=True)
    
    
def download_dem(val, df, epsg = EPSG, ):
    location = calc_bbox(val, df = df)
    bbox = bounding_box(location[0], expansion = (IMSIZE+2)*10)
    box = BBox(bbox, crs = epsg)
    dem_request = WmsRequest(data_source=DataSource.DEM,
                         layer='DEM',
                         bbox=box,
                         width=IMSIZE+2,
                         height=IMSIZE+2,
                         instance_id=API_KEY,
                         image_format=MimeType.TIFF_d32f,
                         custom_url_params={CustomUrlParam.SHOWLOGO: False})
    dem_image = dem_request.get_data()[0]
    dem_image = calcSlope(dem_image.reshape((1, IMSIZE+2, IMSIZE+2)),
                  np.full((IMSIZE+2, IMSIZE+2), 10), np.full((IMSIZE+2, IMSIZE+2), 10), zScale = 1, minSlope = 0.02)
    dem_image = dem_image.reshape((IMSIZE+2, IMSIZE+2, 1))
    dem_image = dem_image[1:IMSIZE+1, 1:IMSIZE+1, :]
    return dem_image

def check_zenith(bbox, epsg = EPSG, time = time):
    try:
        box = BBox(bbox, crs = epsg)
        zenith = WmsRequest(
            layer='ZENITH',
            bbox=box,
            time=time,
            width=IMSIZE,
            height=IMSIZE,
            image_format = MimeType.TIFF_d32f,
            maxcc=0.75,
            instance_id=API_KEY,
            custom_url_params = {constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
            time_difference=datetime.timedelta(hours=48),
        )
        
        zenith = zenith.get_data()
        return zenith
    except Exception as e:
        logging.fatal(e, exc_info=True)
        
def download_layer(bbox, epsg = EPSG, time = time):
    try:
        box = BBox(bbox, crs = epsg)
        image_request = WcsRequest(
                layer='L2A20',
                bbox=box,
                time=time,
                image_format = MimeType.TIFF_d32f,
                maxcc=0.75,
                resx='10m', resy='10m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'NEAREST',
                                    constants.CustomUrlParam.UPSAMPLING: 'NEAREST'},
                time_difference=datetime.timedelta(hours=48),
            )
        img_bands = image_request.get_data()
        img_20 = np.stack(img_bands)
        img_20 = resize(img_20, (img_20.shape[0], IMSIZE, IMSIZE, img_20.shape[-1]), order = 0)
        
        image_request = WcsRequest(
                layer='L2A10',
                bbox=box,
                time=time,
                image_format = MimeType.TIFF_d32f,
                maxcc=0.75,
                resx='10m', resy='10m',
                instance_id=API_KEY,
                custom_url_params = {constants.CustomUrlParam.DOWNSAMPLING: 'BICUBIC',
                                    constants.CustomUrlParam.UPSAMPLING: 'BICUBIC'},
                time_difference=datetime.timedelta(hours=48),
        )
        
        img_bands = image_request.get_data()
        img_10 = np.stack(img_bands)
        print("Original size: {}".format(img_10.shape))
        img_10 = resize(img_10, (img_10.shape[0], IMSIZE, IMSIZE, img_10.shape[-1]), order = 0)
        img = np.concatenate([img_10, img_20], axis = -1)
        return img, image_request

    except Exception as e:
        logging.fatal(e, exc_info=True)

# Cloud and shadow removal

In [84]:
def remove_cloud_and_shadows(tiles, probs, shadows, image_dates, wsize = 5):
    c_probs = np.copy(probs)
    c_probs = c_probs - np.min(c_probs, axis = 0)
    c_probs[np.where(c_probs > 0.33)] = 1.
    c_probs[np.where(c_probs < 0.33)] = 0.
    c_probs = np.reshape(c_probs, [c_probs.shape[0], int(IMSIZE/8), 8, int(IMSIZE/8), 8])
    c_probs = np.sum(c_probs, (2, 4))
    c_probs = resize(c_probs, (c_probs.shape[0], IMSIZE, IMSIZE), 0)
    c_probs[np.where(c_probs < 12)] = 0.
    c_probs[np.where(c_probs >= 12)] = 1.
    c_probs += shadows
    c_probs[np.where(c_probs >= 1.)] = 1.
    n_interp = 0
    for cval in range(0, IMSIZE - 4, 1):
        for rval in range(0, IMSIZE - 4, 1):
            subs = c_probs[:, cval:cval + wsize, rval:rval+wsize]
            satisfactory = [x for x in range(c_probs.shape[0]) if np.sum(subs[x, :, :]) < 10]
            satisfactory = np.array(satisfactory)
            for date in range(0, tiles.shape[0]):
                if np.sum(subs[date, :, :]) > 12:
                    n_interp += 1
                    before, after = calculate_proximal_steps_index(date, satisfactory)
                    before = date + before
                    after = date + after
                    bef = tiles[before, cval:cval+wsize, rval:rval+wsize, : ]
                    aft = tiles[after, cval:cval+wsize, rval:rval+wsize, : ]
                    before = image_dates[before]
                    after = image_dates[after]
                    before_diff = abs(image_dates[date] - before)
                    after_diff = abs(image_dates[date] - after)
                    bef_wt = 1 - before_diff / (before_diff + after_diff)
                    aft_wt = 1 - bef_wt
                    candidate = bef_wt*bef + aft_wt*aft
                    candidate = candidate*c_arr + tiles[date, cval:cval+wsize, rval:rval+wsize, : ]*o_arr
                    tiles[date, cval:cval+wsize, rval:rval+wsize, : ] = candidate  
    print("Interpolated {} px".format(n_interp))
    return tiles

def remove_missed_clouds(img):
    iqr = np.percentile(img[:, :, :, 3].flatten(), 75) - np.percentile(img[:, :, :, 3].flatten(), 25)
    thresh_t = np.percentile(img[:, :, :, 3].flatten(), 75) + iqr*2
    thresh_b = np.percentile(img[:, :, :, 3].flatten(), 25) - iqr*2
    diffs_fw = np.diff(img, 1, axis = 0)
    diffs_fw = np.mean(diffs_fw, axis = (1, 2, 3))
    diffs_fw = np.array([0] + list(diffs_fw))
    diffs_bw = np.diff(np.flip(img, 0), 1, axis = 0)
    diffs_bw = np.flip(np.mean(diffs_bw, axis = (1, 2, 3)))
    diffs_bw = np.array(list(diffs_bw) + [0])
    diffs = abs(diffs_fw - diffs_bw) * 100 # 3, -3 -> 6, -3, 3 -> 6, -3, -3
    #diffs = [int(x) for x in diffs]
    outlier_percs = []
    for step in range(img.shape[0]):
        bottom = len(np.argwhere(img[step, :, :, 3].flatten() > thresh_t))
        top = len(np.argwhere(img[step, :, :, 3].flatten() < thresh_b))
        p = 100* ((bottom + top) / (IMSIZE*IMSIZE))
        outlier_percs.append(p)
    to_remove = np.argwhere(np.array(outlier_percs) > 15)
    print([int(x) for x in outlier_percs])
    return to_remove

# Data interpolation

In [85]:
def calculate_and_save_best_images(img_bands, image_dates):
    # This function interpolates data to 5 day windows linearly

    biweekly_dates = [day for day in range(0, 360, 5)] # ideal imagery dates are every 15 days
    
    # Clouds have been removed at this step, so all steps are satisfactory
    satisfactory_ids = [x for x in range(0, img_bands.shape[0])]
    satisfactory_dates = [value for idx, value in enumerate(image_dates) if idx in satisfactory_ids]
    
    
    selected_images = {}
    for i in biweekly_dates:
        distances = [abs(date - i) for date in satisfactory_dates]
        closest = np.min(distances)
        closest_id = np.argmin(distances)
        # If there is imagery within 5 days, select it
        if closest < 8:
            date = satisfactory_dates[closest_id]
            image_idx = int(np.argwhere(np.array(image_dates) == date)[0])
            selected_images[i] = {'image_date': [date], 'image_ratio': [1], 'image_idx': [image_idx]}
        # If there is not imagery within 7 days, look for the closest above and below imagery
        else:
            distances = np.array([(date - i) for date in satisfactory_dates])
            # Number of days above and below the selected date of the nearest clean imagery
            above = distances[np.where(distances < 0, distances, -np.inf).argmax()]
            below = distances[np.where(distances > 0, distances, np.inf).argmin()]
            if abs(above) > 240: # If date is the last date, occassionally argmax would set above to - number
                above = below
            if abs(below) > 240:
                below = above
            if above != below:
                below_ratio = above / (above - below)
                above_ratio = 1 - below_ratio
            else:
                above_ratio = below_ratio = 0.5
                
            # Extract the image date and imagery index for the above and below values
            above_date = i + above
            above_image_idx = int(np.argwhere(np.array(image_dates) == above_date)[0])
            
            below_date = i + below
            below_image_idx = int(np.argwhere(np.array(image_dates) == below_date)[0])
            
            selected_images[i] = {'image_date': [above_date, below_date], 'image_ratio': [above_ratio, below_ratio],
                                 'image_idx': [above_image_idx, below_image_idx]}
                               
    max_distance = 0
    
    for i in selected_images.keys():
        #print(i, selected_images[i])
        if len(selected_images[i]['image_date']) == 2:
            dist = selected_images[i]['image_date'][1] - selected_images[i]['image_date'][0]
            if dist > max_distance:
                max_distance = dist
    
    print("Maximum time distance: {}".format(max_distance))
        
    # Compute the weighted average of the selected imagery for each time step
    keep_steps = []
    use_median = False
    for i in selected_images.keys():
        step1_additional = None
        step2_additional = None
        info = selected_images[i]
        if len(info['image_idx']) == 1:
            step = img_bands[info['image_idx'][0]]
        if len(info['image_idx']) == 2:
            step1 = img_bands[info['image_idx'][0]] # * info['image_ratio'][0]
            step2 = img_bands[info['image_idx'][1]]
            #if info['image_idx'][0] == 0:
            #    step1_additional = img_bands[-1]
            #    print("Using last step")
            #if info['image_idx'][1] == (img_bands.shape[0] - 1):
            #    step2_additional = img_bands[0]
            ##    print("Using first step")
            #if step1_additional is None and step2_additional is None:
            step = step1 * 0.5 + step2 * 0.5
            #if step1_additional is not None:
            #    print("Echo")
            #    step = (step1 + step2 + step1_additional) * (1/3)
            #if step2_additional is not None:
             #   print("Echo")
            #    step = (step1 + step2 + step2_additional) * (1/3)
        keep_steps.append(step)
    '''
    for i in selected_images.keys():
        info = selected_images[i]
        if len(info['image_idx']) == 1:
            step = img_bands[info['image_idx'][0]]
            use_median = False
        if len(info['image_idx']) == 2:
            difference = np.max([abs(info['image_date'][0] - int(i)),
                                 abs(info['image_date'][1] - int(i))]) 
            step1 = img_bands[info['image_idx'][0]] # * info['image_ratio'][0]
            step2_idx = info['image_idx'][0] - 1
            if step2_idx < 0:
                step2_idx = (img_bands.shape[0] - 1)
            step2 = img_bands[step2_idx]
            step3 = img_bands[info['image_idx'][1]]
            step4_idx = info['image_idx'][1] + 1
            if step4_idx > (img_bands.shape[0] - 1):
                step4_idx = 0
            step4 = img_bands[step4_idx]
            #step2 = img_bands[info['image_idx'][1]] * 0.5 # info['image_ratio'][1]
            if difference > 100 or use_median == True:
                print("Median, {}".format(difference))
                use_median = True
                stacked = np.stack([step1, step2, step3, step4])
                step = np.median(stacked, axis = 0)
            else:
                use_median = False
                step = step1 * 0.5 + step3 * 0.5
        '''
        #keep_steps.append(step)
        
    keep_steps = np.stack(keep_steps)
    return keep_steps, max_distance

# Download function

In [86]:
days_per_month = [0, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30]
starting_days = np.cumsum(days_per_month)
print(starting_days)

[  0  31  59  90 120 151 181 212 243 273 304 334]


In [109]:
super_resolve = True
year = 2019

DATA_LOCATION = '../data/ghana-test.csv'
OUTPUT_FOLDER = '../data/test-smooth-200/'


def download_plots(df, output_folder = OUTPUT_FOLDER):
    plot_ids = [x for x in range(0, len(df))][1500:1650]
    existing = [int(x[:-4]) for x in os.listdir(output_folder) if ".DS" not in x]
    to_download = [x for x in plot_ids if x not in existing]
    print("STARTING DOWNLOAD OF {} plots to {}".format(len(to_download), output_folder))
    errors = []
    for i, val in enumerate(to_download):
        print(i, val)
        print("Downloading {}/{}, {}".format(i+1, len(to_download), val))
        location = calc_bbox(val, df = df)
        location = bounding_box(location[0], expansion = IMSIZE*10)
        location_clouds = bounding_box(location[0], expansion = 96*10)
        try:
            # Identify cloud steps, download DEM, and download L2A series
            probs, shadows, shadow_original, prob_original = identify_clouds(location_clouds)
            shadow_sums = np.sum(shadows, axis = (1, 2))
            shadow_steps = np.argwhere(shadow_sums > (48*48/3))
            dem = download_dem(val, df = df)
            img, image_request = download_layer(location)

            # Subset zenith < 70
            zenith = check_zenith(location)
            zenith = np.mean(np.stack(zenith), axis = (1, 2))
            zenith_outliers = np.argwhere(zenith > 70)
            if len(zenith_outliers) > 0:
                print("Zenith outlier: {}".format(zenith_outliers))

            # Calculate imagery dates
            image_dates = []
            for date in image_request.get_dates():
                if date.year == year - 1:
                    image_dates.append(-365 + starting_days[(date.month-1)] + date.day)
                if date.year == year:
                    image_dates.append(starting_days[(date.month-1)] + date.day)
                if date.year == year + 1:
                    image_dates.append(365 + starting_days[(date.month-1)]+date.day)
            image_dates = np.array(image_dates)
            print(image_dates)

            # Remove imagery where >4% is clouds, and where there is null data
            args = np.array([len(np.argwhere(probs[x].flatten() > 0.3)) for x in range(probs.shape[0])])
            dirty_steps = np.argwhere(args > (IMSIZE)*(IMSIZE) / 5)
            missing_images = [np.argwhere(img[x, :, : :].flatten() == 0.0) for x in range(img.shape[0])]
            missing_images = np.array([len(x) for x in missing_images])
            missing_images_p = [np.argwhere(img[x, :, : :].flatten() >= 1) for x in range(img.shape[0])]
            missing_images_p = np.array([len(x) for x in missing_images_p])
            missing_images += missing_images_p
            missing_images = list(np.argwhere(missing_images >= 25))
            to_remove = np.unique(np.array(list(dirty_steps) + list(missing_images) + list(zenith_outliers) + list(shadow_steps)))

            # Remove null steps
            print("There are {}/{} dirty steps: {} cloud, {} missing, {} zenith {} shadow".format(len(to_remove),
                                                                                        len(img), len(dirty_steps),
                                                                                        len(missing_images),
                                                                                        len(zenith_outliers),
                                                                                       len(shadow_steps)))

            img = np.delete(img, to_remove, 0)
            probs = np.delete(probs, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)

            to_remove = remove_missed_clouds(img)
            img = np.delete(img, to_remove, 0)
            probs = np.delete(probs, to_remove, 0)
            image_dates = np.delete(image_dates, to_remove)
            shadows = np.delete(shadows, to_remove, 0)
            print("Removing {} steps based on ratio".format(len(to_remove)))


            # Concatenate DEM
            dem = np.tile(dem.reshape((1, IMSIZE, IMSIZE, 1)), (img.shape[0], 1, 1, 1))
            tiles = np.concatenate([img, dem], axis = -1)
            tiles[:, :, :, -1] /= 90

            x = remove_cloud_and_shadows(tiles, probs, shadows, image_dates)
            if super_resolve:
                x = x[:, 8:40, 8:40, :]
                print("Before super: {}".format(x.shape))

                d10 = x[:, :, :, 0:4]
                d20 = x[:, :, :, 4:10]

                d10 = np.swapaxes(d10, 1, -1)
                d10 = np.swapaxes(d10, 2, 3)
                d20 = np.swapaxes(d20, 1, -1)
                d20 = np.swapaxes(d20, 2, 3)
                superresolved = DSen2(d10, d20)
                superresolved = np.swapaxes(superresolved, 1, -1)
                superresolved = np.swapaxes(superresolved, 1, 2)
                print(superresolved.shape)
                print(x.shape)

                # returns band IDXs 3, 4, 5, 7, 8, 9
                x[:, :, :, 4:10] = superresolved
                x = x[:, 8:24, 8:24, :]
                print("After super shape: {}".format(x.shape))
            else:
                bottom = int(IMSIZE/2 - 8)
                top = int(IMSIZE/2 + 8)
                x = x[:, bottom:top, bottom:top, :]

            # Calculate indices
            tiles, amin = evi(x, True)
            # Where evi is OOB, remove (likely cloud cover missed)
            #if len(amin) > 0:
            #    satisfactory = [x for x in range(tiles.shape[0]) if x not in amin]
            ##    for i in amin:
            #        before, after = calculate_proximal_steps_index(i, satisfactory)
            #        print("Interpolating {} with {} and {}".format(i, before, after))
            #        bef = tiles[before, :, :, :]
            #        aft = tiles[after, :, :, :]
            #        tiles[i, :, :, :] = (bef + aft) / 2

            tiles = bi(tiles, True)
            tiles = msavi2(tiles, True)
            x = si(tiles, True)

            print("Clean: {}".format(x.shape))
            
            missing_pixels = 0
            for band in range(0, 15):
                for time in range(0, x.shape[0]):
                    x_i = x[time, :, :, band]
                    missing_pixels += len(np.argwhere(np.isnan(x_i)))
                    x_i[np.argwhere(np.isnan(x_i))] = np.mean(x_i)
                    x[time, :, :, band] = x_i
            print("There are {} missing pixels".format(missing_pixels))

            # Interpolate linearly to 5 day frequency
            tiles, max_distance = calculate_and_save_best_images(x, image_dates) # 22, 16, 16, 10

            # Smooth linear interpolation
            for row in range(0, 16):
                for column in range(0, 16):
                    for band in [x for x in range(0, 15) if x != 10]:
                        sm = smooth(tiles[:, row, column, band], 800, d = 2)
                        tiles[:, row, column, band] = sm

            # Retain only iamgery every 15 days
            biweekly_dates = np.array([day for day in range(0, 360, 5)])
            to_remove = np.argwhere(biweekly_dates % 15 != 0)
            tiles = np.delete(tiles, to_remove, 0)
            print(tiles.shape)

            if max_distance <= 240:
                np.save(output_folder + str(val), tiles)
                print("\n")
            else:
                print("Skipping {} because there is a {} distance".format(val, max_distance))
                print("\n")

        except Exception as e:
            print(e)
            logging.fatal(e, exc_info=True)
            errors.append(img)
            #continue

In [110]:
x = np.load("../data/train-super/136089019.npy")
sns.heatmap(x[12, :, :, 3])

FileNotFoundError: [Errno 2] No such file or directory: '../data/train-super/136089019.npy'

In [111]:
import math
download_plots(df, "../../data/fra-s2-stratified-new/")

STARTING DOWNLOAD OF 150 plots to ../../data/fra-s2-stratified-new/
0 1500
1500
27.209921699454203 -21.662290315485176
[-235. -235.]
[245. 245.]
27.20765341985707 -21.66441636745994
[-235. -235.]
[245. 245.]
(66, 96, 96, 10)
1
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
The shadow probability is: 0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
The shadow probability is: 0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
1503
28.25196839152831 -19.526815484985434
[-235. -235.]
[245. 245.]
Original size: (71, 49, 48, 4)
[-15 -10  -5   5  10  25  30  35  40  45  50  55  60  65  70  80  85  90
  95 100 105 110 115 120 125 130 135 140 145 150 155 160 165 170 175 185
 190 195 200 205 210 215 220 225 230 235 240 245 250 255 260 265 270 275
 280 285 290 295 300 305 310 315 320 335 340 345 350 355 360 365 370]
There are 11/71 dirty steps: 11 cloud, 1 missing, 0 zenith 0 shadow
[0, 0, 0, 8, 0, 63, 0, 0, 0, 0, 0, 0, 0, 0, 45, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

CRITICAL:root:cannot do a non-empty take from an empty axes.
Traceback (most recent call last):
  File "<ipython-input-109-fa25f080e790>", line 70, in download_plots
    to_remove = remove_missed_clouds(img)
  File "<ipython-input-84-b450c09bc277>", line 40, in remove_missed_clouds
    iqr = np.percentile(img[:, :, :, 3].flatten(), 75) - np.percentile(img[:, :, :, 3].flatten(), 25)
  File "/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/numpy/lib/function_base.py", line 3707, in percentile
    a, q, axis, out, overwrite_input, interpolation, keepdims)
  File "/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/numpy/lib/function_base.py", line 3826, in _quantile_unchecked
    interpolation=interpolation)
  File "/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.6/site-packages/numpy/lib/function_base.py", line 3405, in _ureduce
    r = func(a, **kwargs)
  File "/Users/john.brandt/anaconda3/envs/remote_sensing/lib/python3.

[-15 -12  -7  -2   3   8  15  18  23  30  33  38  43  48  55  60  63  68
  73  78  83  88  93  98 103 108 113 120 123 128 133 138 143 148 155 158
 163 168 173 178 183 188 193 198 203 208 213 218 223 228 233 238 243 248
 253 258 263 268 273 278 283 288 295 298 303 308 313 318 325 328 333 338
 345 348 353 358 363 368 373 378]
There are 80/80 dirty steps: 80 cloud, 0 missing, 0 zenith 0 shadow
cannot do a non-empty take from an empty axes.
5 1505
1505
8.830393496171045 11.024370754646464
[-235. -235.]
[245. 245.]
8.828243410984285 11.022244215330344
[-235. -235.]
[245. 245.]
(62, 96, 96, 10)
58
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0. 270.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.  10.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0.   0.   0.   0.]
[  0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   0.
   0.   0.   0

Interpolated 403 px
Before super: (28, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
28/28 [==============================] - 1s 29ms/step
(28, 32, 32, 6)
(28, 32, 32, 11)
After super shape: (28, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (28, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 28
(24, 16, 16, 15)


8 1508
1508
-7.500978758541418 11.850140041069038
[-235. -235.]
[245. 245.]
-7.50314715897319 11.848026960617762
[-235. -235.]
[245. 245.]
(62, 96, 96, 10)
54
[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 1.100e+02 8.000e+00
 6.400e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 6.800e+01 1.635e+03 0.000e+00 0.000e+00 0.000e+00 3.401e+03 8.000e+00
 4.796e+03 0.000e+00 3.600e+01 2.000e+00 3.320e

...

Interpolated 2315 px
Before super: (38, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
38/38 [==============================] - 1s 29ms/step
(38, 32, 32, 6)
(38, 32, 32, 11)
After super shape: (38, 16, 16, 11)
There are: 0 out of bounds EVI
Clean: (38, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 30
(24, 16, 16, 15)


115 1615
1615
-103.08173663859431 37.56300687717557
[-235. -235.]
[245. 245.]
-103.08445027576188 37.56093307008527
[-235. -235.]
[245. 245.]
(75, 96, 96, 10)
34
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
[0. 0. 0.

(74, 96, 96, 10)
18
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
The shadow probability is: 0.0
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]
1618
-113.91544171292291 57.8555562127570

Original size: (66, 47, 49, 4)
[-11  -1   4   9  14  19  34  39  44  49  54  59  64  69  74  79  84  89
  94  99 119 124 129 134 139 144 149 154 159 164 169 174 179 184 189 194
 199 204 209 214 224 229 234 239 244 249 254 259 264 269 274 284 289 294
 299 304 309 329 334 339 344 349 359 369 374 379]
There are 20/66 dirty steps: 19 cloud, 3 missing, 0 zenith 1 shadow
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 5, 0, 0, 0, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 3, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Removing 0 steps based on ratio
Interpolated 1072 px
Before super: (46, 32, 32, 11)
Predicting using file: ../../src/dsen2/models/s2_032_lr_1e-04.hdf5
46/46 [==============================] - 1s 31ms/step
(46, 32, 32, 6)
(46, 32, 32, 11)
After super shape: (46, 16, 16, 11)
There are: 0 out of bounds EVI
()
evis error: 0.014282813295722008, 1.032035231590271, 0 steps, clipping to -1.5, 1.5
Clean: (46, 16, 16, 15)
There are 0 missing pixels
Maximum time distance: 30
(24, 16, 16, 15)


122 1

(24, 16, 16, 15)


125 1625
1625
18.6579593980736 -21.762811011774215
[-235. -235.]
[245. 245.]
18.655653923503664 -21.764900225438634
[-235. -235.]
[245. 245.]
(82, 96, 96, 10)
10
[ 0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0. 16.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]
[ 0.  0.  0.  0. 12.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0. 

KeyboardInterrupt: 